In [54]:
!pip install optuna
!pip install torchmetrics

In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pandas as pd
import optuna
import torchmetrics

In [56]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [57]:
matches = pd.read_csv('matches_6_years.csv', index_col = 0)

In [58]:
matches.head()

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2022-08-07,16:30,Premier League,Matchweek 1,Sun,Away,W,2,0,West Ham,...,Match Report,NaN,13.0,1.0,18.7,1.0,1,1,2022-2023,Manchester City
2,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4,0,Bournemouth,...,Match Report,NaN,19.0,7.0,17.5,0.0,0,0,2022-2023,Manchester City
3,2022-08-21,16:30,Premier League,Matchweek 3,Sun,Away,D,3,3,Newcastle Utd,...,Match Report,NaN,21.0,10.0,16.2,1.0,0,0,2022-2023,Manchester City
4,2022-08-27,15:00,Premier League,Matchweek 4,Sat,Home,W,4,2,Crystal Palace,...,Match Report,NaN,18.0,5.0,14.1,0.0,0,0,2022-2023,Manchester City
5,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,6,0,Nott'ham Forest,...,Match Report,NaN,17.0,9.0,14.8,0.0,0,0,2022-2023,Manchester City


In [59]:
matches['date']=pd.to_datetime(matches['date'])

matches['venue_code'] = matches['venue'].astype('category').cat.codes

matches['opp_code'] = matches['opponent'].astype('category').cat.codes

matches['hour'] = matches['time'].str.replace(':.+','',regex=True).astype('int')

matches['day_code']= matches['date'].dt.dayofweek

matches['target']=(matches['result'] == 'W')

In [60]:
test = matches[matches['date'] > '2022-01-01']

train = matches[matches['date'] < '2022-01-01']

predictors = ['venue_code', 'opp_code', 'hour', 'day_code']

X_train = train[predictors]

X_test = test[predictors]

y_test= test ['target']

y_train = train['target']

In [61]:
predictors

['venue_code', 'opp_code', 'hour', 'day_code']

In [62]:
X_train_subset = train[['venue_code', 'opp_code', 'hour', 'day_code',"gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt" ]]
X_test_subset = test[['venue_code', 'opp_code', 'hour', 'day_code',"gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt" ]]

# Displaying the DataFrame
print(X_train_subset.head())
print(X_test_subset.head())

   venue_code  opp_code  hour  day_code  gf  ga    sh   sot  dist   fk  pk  \
1           0        24    16         6   0   1  18.0   4.0  17.3  1.0   0   
2           1        18    15         5   5   0  16.0   4.0  18.5  1.0   0   
3           1         0    12         5   5   0  25.0  10.0  14.8  0.0   0   
4           0        13    15         5   1   0  25.0   8.0  14.3  0.0   0   
6           1        21    15         5   0   0  16.0   1.0  16.4  1.0   0   

   pkatt  
1      0  
2      0  
3      0  
4      0  
6      0  
   venue_code  opp_code  hour  day_code  gf  ga    sh   sot  dist   fk  pk  \
1           0        27    16         6   2   0  13.0   1.0  18.7  1.0   1   
2           1         2    15         5   4   0  19.0   7.0  17.5  0.0   0   
3           0        17    16         6   3   3  21.0  10.0  16.2  1.0   0   
4           1         8    15         5   4   2  18.0   5.0  14.1  0.0   0   
5           1        19    19         2   6   0  17.0   9.0  14.8  0.0   0 

In [63]:
X_test_subset.head()

,venue_code,opp_code,hour,day_code,gf,ga,sh,sot,dist,fk,pk,pkatt
1,0,27,16,6,2,0,13.0,1.0,18.7,1.0,1,1
2,1,2,15,5,4,0,19.0,7.0,17.5,0.0,0,0
3,0,17,16,6,3,3,21.0,10.0,16.2,1.0,0,0
4,1,8,15,5,4,2,18.0,5.0,14.1,0.0,0,0
5,1,19,19,2,6,0,17.0,9.0,14.8,0.0,0,0


In [64]:
class NeuralNetwork(nn.Module):
    def __init__(self,n_input, n_hidden):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(n_input, n_hidden)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(n_hidden, n_hidden)
        self.output = nn.Linear(n_hidden, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.sigmoid(self.output(x))
        return x

In [65]:
train_features = torch.Tensor(X_train.values)

In [66]:
train_features

tensor([[ 0., 24., 16.,  6.],
        [ 1., 18., 15.,  5.],
        [ 1.,  0., 12.,  5.],
        ...,
        [ 0., 17., 15.,  5.],
        [ 1., 24., 15.,  5.],
        [ 0.,  8., 15.,  6.]])

In [67]:
def prepare_data_loaders(X_train, y_train, X_test, y_test, batch_size):
    train_features = torch.Tensor(X_train.values)
    train_targets = torch.Tensor(y_train.values).view(-1, 1)
    train_dataset = TensorDataset(train_features, train_targets)
    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

    test_features = torch.Tensor(X_test.values)
    test_targets = torch.Tensor(y_test.values).view(-1, 1)
    test_dataset = TensorDataset(test_features, test_targets)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader

In [68]:
def evaluate_model(model, test_loader):
    model.eval()
    accuracy_metric = torchmetrics.Accuracy(threshold=0.5, task='binary')
    precision_metric = torchmetrics.Precision(num_classes=2, average='macro', threshold=0.5, task='binary')
    recall_metric = torchmetrics.Recall(num_classes=2, average='macro', threshold=0.5, task='binary')
    f1_metric = torchmetrics.F1Score(num_classes=2, average='macro', threshold=0.5, task='binary')

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            predicted = (outputs > 0.5).float()
            accuracy_metric.update(predicted, targets.int())
            precision_metric.update(predicted, targets.int())
            recall_metric.update(predicted, targets.int())
            f1_metric.update(predicted, targets.int())

    accuracy = accuracy_metric.compute()
    precision = precision_metric.compute()
    recall = recall_metric.compute()
    f1_score = f1_metric.compute()

    return accuracy, precision, recall, f1_score

In [69]:
def train_model(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        for inputs, targets in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


In [70]:
def objective(trial):
    # Hyperparameters
    n_hidden_layers = trial.suggest_int('n_hidden', 10, 100)
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])

    # Prepare data loaders
    train_loader, test_loader = prepare_data_loaders(X_train, y_train, X_test, y_test, batch_size)

    # Setup the neural network
    model = NeuralNetwork(n_input = len(predictors), n_hidden=n_hidden_layers)
    criterion = nn.BCELoss()
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Train and evaluate the model
    train_model(model, train_loader, criterion, optimizer)
    accuracy, precision, recall, f1_score = evaluate_model(model, test_loader)
    print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1_score}")

    # Store all metrics in the trial
    trial.set_user_attr('accuracy', accuracy)
    trial.set_user_attr('precision', precision)
    trial.set_user_attr('recall', recall)
    trial.set_user_attr('f1_score', f1_score)

    return accuracy


In [71]:
# Set the seed for reproducibility
sampler = optuna.samplers.TPESampler(seed=10)  # Use the same seed for the random sampler
study = optuna.create_study(direction='maximize', sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2024-04-23 15:52:44,560] A new study created in memory with name: no-name-26cc2e8f-5436-4dc4-9b7f-3385466d9419
<ipython-input-70-659680de4fc3>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
[I 2024-04-23 15:52:45,686] Trial 0 finished with value: 0.44947734475135803 and parameters: {'n_hidden': 80, 'lr': 1.2106198691436028e-05, 'batch_size': 64, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.44947734475135803.


Accuracy: 0.44947734475135803, Precision: 0.3973214328289032, Recall: 0.7946428656578064, F1 Score: 0.5297619104385376


[I 2024-04-23 15:52:46,738] Trial 1 finished with value: 0.6097561120986938 and parameters: {'n_hidden': 25, 'lr': 2.2561047334047238e-05, 'batch_size': 64, 'optimizer': 'RMSprop'}. Best is trial 1 with value: 0.6097561120986938.


Accuracy: 0.6097561120986938, Precision: 0.0, Recall: 0.0, F1 Score: 0.0


[I 2024-04-23 15:52:48,455] Trial 2 finished with value: 0.6097561120986938 and parameters: {'n_hidden': 75, 'lr': 0.00014706328837360436, 'batch_size': 32, 'optimizer': 'SGD'}. Best is trial 1 with value: 0.6097561120986938.


...

Accuracy: 0.6097561120986938, Precision: 0.0, Recall: 0.0, F1 Score: 0.0


[I 2024-04-23 15:54:51,003] Trial 98 finished with value: 0.6088849902153015 and parameters: {'n_hidden': 97, 'lr': 0.0002520431145649149, 'batch_size': 32, 'optimizer': 'RMSprop'}. Best is trial 35 with value: 0.6236934065818787.


Accuracy: 0.6088849902153015, Precision: 0.0, Recall: 0.0, F1 Score: 0.0


[I 2024-04-23 15:54:52,049] Trial 99 finished with value: 0.6097561120986938 and parameters: {'n_hidden': 54, 'lr': 0.00032038182179717356, 'batch_size': 64, 'optimizer': 'Adam'}. Best is trial 35 with value: 0.6236934065818787.


Accuracy: 0.6097561120986938, Precision: 0.0, Recall: 0.0, F1 Score: 0.0


In [72]:
# Print the results for the best trial
trial = study.best_trial

print("Best trial without Rolling Averages:")


accuracy = trial.user_attrs['accuracy']
precision = trial.user_attrs['precision']
recall = trial.user_attrs['recall']
f1_score = trial.user_attrs['f1_score']

print(f"  Value (Accuracy): {trial.value}")
print("  Metrics:")
print(f"    Accuracy: {accuracy}")
print(f"    Precision: {precision}")
print(f"    Recall: {recall}")
print(f"    F1 Score: {f1_score}")
print("  Params: ")
for key, value in trial.params.items():
  print(f"    {key}: {value}")

Best trial without Rolling Averages:
  Value (Accuracy): 0.6236934065818787
  Metrics:
    Accuracy: 0.6236934065818787
    Precision: 0.5634920597076416
    Recall: 0.1584821492433548
    F1 Score: 0.24738675355911255
  Params: 
    n_hidden: 53
    lr: 0.006177441034744258
    batch_size: 128
    optimizer: Adam


In [73]:
trial.params

{'n_hidden': 53,
 'lr': 0.006177441034744258,
 'batch_size': 128,
 'optimizer': 'Adam'}

In [74]:
trial.params['batch_size']

128

In [75]:
## Rolling averages
grouped_matches = matches.groupby("team")
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [76]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [77]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

In [78]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))
matches_rolling = matches_rolling.droplevel('team')
matches_rolling.index = range(matches_rolling.shape[0])

In [79]:
new_train = matches_rolling[matches_rolling["date"]< '2022-01-01']
new_test = matches_rolling[matches_rolling["date"]>'2022-01-01']
new_predictors = predictors + new_cols

In [80]:
X_train_rolling = new_train[new_predictors]
X_test_rolling = new_test[new_predictors]
y_test_rolling=  new_test ['target']
y_train_rolling = new_train['target']

In [81]:
X_train_rolling_subset = new_train[new_predictors]
X_test_rolling_subset = new_test[new_predictors]

# Displaying the DataFrame
print(X_train_rolling_subset.head())
print(X_test_rolling_subset.head())

   venue_code  opp_code  hour  day_code  gf_rolling  ga_rolling  sh_rolling  \
0           1         2    15         5    1.333333    2.666667   17.666667   
1           0         7    13         6    1.000000    1.666667   14.333333   
2           1        26    20         0    1.000000    1.333333   12.000000   
3           1         4    12         6    1.666667    0.000000   14.333333   
4           0        25    17         5    1.333333    0.000000   17.000000   

   sot_rolling  dist_rolling  fk_rolling  pk_rolling  pkatt_rolling  
0     5.333333     18.133333    0.000000    0.000000       0.000000  
1     5.000000     16.766667    0.333333    0.000000       0.000000  
2     3.666667     16.566667    0.333333    0.000000       0.000000  
3     5.333333     17.400000    1.333333    0.333333       0.333333  
4     5.000000     18.333333    1.666667    0.333333       0.333333  
     venue_code  opp_code  hour  day_code  gf_rolling  ga_rolling  sh_rolling  \
169           1         

In [82]:
X_train_rolling_subset.head()

,venue_code,opp_code,hour,day_code,gf_rolling,ga_rolling,sh_rolling,sot_rolling,dist_rolling,fk_rolling,pk_rolling,pkatt_rolling
0,1,2,15,5,1.333333,2.666667,17.666667,5.333333,18.133333,0.000000,0.000000,0.000000
1,0,7,13,6,1.000000,1.666667,14.333333,5.000000,16.766667,0.333333,0.000000,0.000000
2,1,26,20,0,1.000000,1.333333,12.000000,3.666667,16.566667,0.333333,0.000000,0.000000
3,1,4,12,6,1.666667,0.000000,14.333333,5.333333,17.400000,1.333333,0.333333,0.333333
4,0,25,17,5,1.333333,0.000000,17.000000,5.000000,18.333333,1.666667,0.333333,0.333333


In [83]:
# Initialize the network with the params
model = NeuralNetwork(n_input= len(new_predictors), n_hidden=trial.params['n_hidden'])
criterion = nn.BCELoss()
['Adam', 'RMSprop', 'SGD']
if trial.params['optimizer'] == 'RMSprop':
  optimizer = optim.RMSprop(model.parameters(), lr=trial.params['lr'])
elif trial.params['optimizer'] == 'Adam':
  optimizer = optim.Adam(model.parameters(), lr=trial.params['lr'])
else:
  optimizer = optim.SGD(model.parameters(), lr=trial.params['lr'])

In [84]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.006177441034744258
    maximize: False
    weight_decay: 0
)

In [85]:
train_loader_rolling, test_loader_rolling = prepare_data_loaders(X_train_rolling, y_train_rolling, X_test_rolling, y_test_rolling, trial.params['batch_size'])

In [86]:
train_model(model, train_loader_rolling, criterion, optimizer)
accuracy_rolling, precision_rolling, recall_rolling, f1_score_rolling = evaluate_model(model, test_loader_rolling)
print(f" Accuracy_rolling: {accuracy_rolling}\n Precision_rolling: {precision_rolling}\n Recall_rolling: {recall_rolling}\n F1 Score_rolling: {f1_score_rolling}")

 Accuracy_rolling: 0.6445415019989014
 Precision_rolling: 0.568965494632721
 Recall_rolling: 0.3691275119781494
 F1 Score_rolling: 0.447761207818985


# Comparing Results

In [87]:
metrics = {
    'Accuracy': accuracy.item(),
    'F1 Score': f1_score.item(),
    'Recall': recall.item(),
    'Precision': precision.item()
}

metrics_rolling = {
    'Accuracy': accuracy_rolling.item(),
    'F1 Score': f1_score_rolling.item(),
    'Recall': recall_rolling.item(),
    'Precision': precision_rolling.item()
}

In [88]:
df_metrics = pd.DataFrame({
    'Metrics without Rolling Averages': metrics,
    'Metrics with Rolling Averages': metrics_rolling
})
df_metrics

,Metrics without Rolling Averages,Metrics with Rolling Averages
Accuracy,0.623693,0.644542
F1 Score,0.247387,0.447761
Recall,0.158482,0.369128
Precision,0.563492,0.568965


In [89]:
df_metrics = df_metrics.T
df_metrics

,Accuracy,F1 Score,Recall,Precision
Metrics without Rolling Averages,0.623693,0.247387,0.158482,0.563492
Metrics with Rolling Averages,0.644542,0.447761,0.369128,0.568965


In [90]:
df_metrics.round(2)
df_metrics

,Accuracy,F1 Score,Recall,Precision
Metrics without Rolling Averages,0.623693,0.247387,0.158482,0.563492
Metrics with Rolling Averages,0.644542,0.447761,0.369128,0.568965


In [91]:
df_metrics.round(2)

,Accuracy,F1 Score,Recall,Precision
Metrics without Rolling Averages,0.62,0.25,0.16,0.56
Metrics with Rolling Averages,0.64,0.45,0.37,0.57
